In [89]:
import os
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input

## Load Data and normalization

In [84]:
print("[INFO] loading dataset...")

path = os.path.join("dataset", "bounding")

dataset = pd.read_csv("{}/dataset.csv".format(path))

print("[INFO] Load image from path...")
image = []
img_x = []
img_y = []
img_w = []
img_h = []
labels = []

drop_index = []
for index, row in dataset.iterrows():
    img = cv2.imread(row["path"])
    (h, w) = img.shape[:2]
    
    x_result = float(row["x"]) / w
    y_result = float(row["y"]) / h
    w_result = float(row["w"]) / w
    h_result = float(row["h"]) / h
    
    
    img_x.append(1. if x_result > 1 else x_result)
    img_y.append(1. if y_result > 1 else y_result)
    img_w.append(1. if w_result > 1 else w_result)
    img_h.append(1. if h_result > 1 else h_result)
    labels.append(1 if row["label"] == "theft" else 0)
    
    img = load_img(row["path"], target_size=(320, 320))
    img = img_to_array(img)
    image.append(img)
        
print("[INFO] Success load image from path...")
        
boxes = np.array([img_x, img_y, img_w, img_h]).transpose()
images = np.array(image)/255.
labels = np.array(labels)

print("[INFO] Success loading dataset")

[INFO] loading dataset...
[INFO] Load image from path...
[INFO] Success load image from path...
[INFO] Success loading dataset


## Split data training and data validation

In [90]:
split = train_test_split(images, boxes, labels, test_size=0.10, random_state=42)

(train_image, test_image) = split[:2]
(train_box, test_box) = split[2:4]
(train_label, test_label) = split[4:6]

## Model

In [56]:
dataset.describe()

w_img = 224
h_img = 398

print("[INFO] Drop data {} from dataset".format(drop_index))
dataset = dataset.drop(drop_index)
print("[INFO] Success drop data {} from dataset".format(drop_index))

boxes = pd.concat([
    dataset["x"].astype(float)/w_img,
    dataset["y"].astype(float)/h_img,
    dataset["w"].astype(float)/w_img,
    dataset["x"].astype(float)/h_img
])

if h_img != 398 and w_img != 224:
        drop_index.append(index)

,x,y,w,h
count,1041.000000,1041.000000,1041.000000,1041.000000
mean,210.788665,115.449568,83.294909,210.788665
std,86.133168,42.690001,34.599584,86.133168
min,19.000000,0.000000,10.000000,19.000000
25%,166.000000,89.000000,65.000000,166.000000
50%,219.000000,120.000000,82.000000,219.000000
75%,270.000000,138.000000,103.000000,270.000000
max,385.000000,340.000000,203.000000,385.000000


In [ ]:
for index, row in dataset.iterrows():
    img = cv2.imread(row["path"])
    (h, w) = img.shape[:2]
    print(img.shape)

In [16]:
dataset

,path,x,y,w,h,label
0,dataset/original/guest/ts1 (348).jpeg,211,138,125,211,guest
1,dataset/original/guest/ts1 (129).jpeg,306,74,83,306,guest
2,dataset/original/guest/ts1 (92).jpeg,135,134,34,135,guest
3,dataset/original/guest/ts1 (266).jpeg,302,82,94,302,guest
4,dataset/original/guest/ts1 (441).jpeg,189,120,83,189,guest
...,...,...,...,...,...,...
1036,dataset/original/theft/M (94).jpeg,159,214,72,159,theft
1037,dataset/original/theft/M (106).jpeg,385,11,149,385,theft
1038,dataset/original/theft/sm (64).jpeg,252,105,68,252,theft
1039,dataset/original/theft/M (46).jpeg,40,110,17,40,theft


In [5]:
mobile_net = MobileNetV2(
    weights = "imagenet", 
    include_top = False,
    input_tensor = Input(shape = (320, 320, 3), name = "image")
)

for layer in mobile_net.layers:
    layer.trainable = False

NameError: name 'Input' is not defined